# oceanparcels_demo.ipynb (mostly from Dan Bartley)

All the material below is taken from the tutorial notebooks at https://oceanparcels.org/ - I've just picked out stuff from there.

* NOTE: Run the below code first to download some packages and driver files.

In [ ]:
%pip install parcels cgen cftime netCDF4 zarr pymbolic
%pip install cartopy
# !wget https://zenodo.org/record/8143129/files/summer_2011.nc
# !wget https://zenodo.org/record/8143129/files/winter_2010.nc

## Import packages

In [ ]:
%matplotlib inline
from parcels import FieldSet, ParticleSet, JITParticle, AdvectionRK4, plotTrajectoriesFile, ErrorCode
from datetime import timedelta
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

## Load velocities from NetCDF files
These are loaded into an OceanParcels FieldSet object.

In [ ]:
# Names of files containing the data
velocity_file = "winter_2010.nc"
filenames = {
    'U': velocity_file,
    'V': velocity_file
}

# Names of the velocity component data in the file
variables = {
    'U': 'uu',
    'V': 'vv'
}

# Names of the dimensions in the file
dimensions = {
    'lat': 'latc',
    'lon': 'lonc',
    'time': 'time'
}

# Create the FieldSet object
fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)

## Initialise particles
This is done using a ParticleSet object. I've included how it might be done for 1) putting individual one or few particles, and 2) fun particle shapes

#### 1) Putting one or a few particles

In [ ]:
# Initial positions of particles
# (one line per particle, add/remove lines as desired)
particle_positions = np.array([
    [113.7, 22.6], # (longitude, latitude)
    [113.71, 22.6], # (longitude, latitude)
    # ...
])

# Release times of particles (must have a release time per particle)
# Alternatively if you don't include time they get released at the first time value in the velocity file
particle_times = np.arange(2)*timedelta(hours=8).total_seconds()

# Create the ParticleSet
pset = ParticleSet.from_list(
    fieldset=fieldset,
    pclass=JITParticle,
    lon=particle_positions[:, 0],
    lat=particle_positions[:, 1],
    time=particle_times
)

#### 2) Fun particle shapes
I have included some fun designs for particle positions (smiley face, rubber duck, HKUST "logo"). These have integer coordinates for rows and columns and are centred at (0, 0). To place them in the domain, just add on the desired centre and multiply by a desired scale factor. For these I didn't set any release time so they are just released at the first time value in the velocity file.

In [ ]:
# Load the coordinates
coords = np.loadtxt(f"fun_particle_shapes/smiley_face.csv", delimiter=",", dtype=int)

# Move the shape to the desired centre and multiply by a scale factor
particle_positions = np.array([114.62, 22.1]) + (0.0075*coords)

# Create the ParticleSet
pset = ParticleSet.from_list(
    fieldset=fieldset,
    pclass=JITParticle,
    lon=particle_positions[:, 0],
    lat=particle_positions[:, 1]
)

#### Show the particle initial positions
The `domain` argument specifies the maximum extent of the plot otherwise it defaults to the minimum boundary that covers the particles which isn't super helpful. Note that if you want to change this domain make sure it's within the range of the velocity data otherwise it will throw an error.

In [ ]:
# code seems to crash the kernel, maybe don't use this...

# pset.show(
#     domain={
#         "N":23.00,
#         "S":21.75,
#         "E":114.95,
#         "W":112.95
#     }
# )

## Simulate particle motion
Trajectories are saved to a `.zarr` file which can be reopened later for plotting.

In [ ]:
# Length of time to simulate
runtime = timedelta(days=5)

# Integration timestep
dt = timedelta(minutes=5)

# Name of output file
trajectory_filename = "particle_trajectories.zarr"

# Frequency of outputs
output_frequency = timedelta(minutes=30)

# Function for deleting particles that go out of the boundary
def DeleteParticle(particle, fieldset, time):
    particle.delete()

# Simulate and save the trajectories
output_file = pset.ParticleFile(name=trajectory_filename, outputdt=output_frequency)
pset.execute(
    AdvectionRK4,
    runtime=runtime,
    dt=dt,
    output_file=output_file,
    recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle}
)

# Plotting

I think OceanParcels doesn't really have its own plotting functionality, so they recommend you write your own. The code below comes from the animation tutorial in https://nbviewer.org/github/OceanParcels/parcels/blob/master/parcels/examples/tutorial_output.ipynb. This particular way of doing it seems to be painfully slow in some cases e.g. when there are particles released at different times, so an alternative option may be needed, which may also depend on what platform you're using (e.g. Jupyter/Colab)

(Note: as a quick and dirty method you can use the OceanParcels function `plotTrajectoriesFile("filename.zarr",mode="movie2d_notebook")`, but it is very basic and doesn't seem to behave very well when using the method to delete particles that go outside the boundary, so probably best to stick to custom-made options)

In [ ]:
# Load the trajectory data
trajectory_filename = "particle_trajectories.zarr"
data_xarray = xr.open_zarr(trajectory_filename)

dt = np.timedelta64(timedelta(minutes=30))

# Times at which to plot
timerange = np.arange(
    np.nanmin(data_xarray['time'].values),
    np.nanmax(data_xarray['time'].values) + dt,
    dt
)

### 1) Run the two cells below for in window animation (first creates object and second animates it)

In [ ]:
# in window animation (run the next cell)

%%capture

# Initialise the figure
fig = plt.figure()
ax = fig.add_subplot(projection=ccrs.PlateCarree())

# Initial time ID
time_id = np.where(data_xarray['time'] == timerange[0])

# Initial plot positions
scatter = ax.scatter(data_xarray['lon'].values[time_id], data_xarray['lat'].values[time_id])

# Graph settings
t = str(timerange[0])
title = ax.set_title("Particle positions at \n"+t)
ax.set_xlabel('Longitude [°E]')
ax.set_ylabel('Latitude [°N]')
ax.set_xlim(112.95, 114.95)
ax.set_ylim(21.75, 23)
ax.set_xticks(np.linspace(113, 114.5, 4))
ax.set_yticks(np.linspace(22, 23, 3))
ax.coastlines()
ax.grid()

def animate(i):

    # Update title
    t = str(timerange[i])
    title.set_text("Particle positions at\n"+t)

    # Update scatter positions
    time_id = np.where(data_xarray['time'] == timerange[i])
    scatter.set_offsets(np.c_[data_xarray['lon'].values[time_id], data_xarray['lat'].values[time_id]])

anim = FuncAnimation(fig, animate, frames=len(timerange), interval=500)

In [ ]:
HTML(anim.to_jshtml())

### 2) Run the cell below for a fancy version but outputting the raw frame files (and you can throw those into an animator of your choice, e.g. on the internet)

In [ ]:
# fancy plot with more cartopy functionalities

from cartopy.io import shapereader
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.io.img_tiles as cimgt

def make_map(projection=ccrs.PlateCarree()):
    fig, ax = plt.subplots(figsize=(8, 6),
                           subplot_kw=dict(projection=projection))
    gl = ax.gridlines(draw_labels=True)
    gl.top_labels_top = gl.right_labels = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    return fig, ax
import cartopy.io.img_tiles as cimgt

extent = [112.95, 114.95, 21.75, 23]

request = cimgt.GoogleTiles() # style="satellite" for satellite image; don't make extent too big

fig, ax = make_map(projection=request.crs)
ax.set_extent(extent)

ax.add_image(request, 10, interpolation='spline36', zorder=-5)

# actual plot of the scatter

i = 0

# Initial time ID
time_id = np.where(data_xarray['time'] == timerange[0])

# Initial plot positions
scatter = ax.scatter(data_xarray['lon'].values[time_id], data_xarray['lat'].values[time_id],
                     transform=ccrs.PlateCarree(), color="k")

t = str(timerange[0])
title = ax.set_title("Particle positions at \n"+t)

plt.savefig(f"./video/position_{i:04}.png", dpi=75, bbox_inches="tight")

for i in range(len(timerange)):

    if i % 10 == 0:
        print(f"working on frame number {i} / {len(timerange)}")

    # Update title
    t = str(timerange[i])
    title.set_text("Particle positions at\n"+t)

    time_id = np.where(data_xarray['time'] == timerange[i])
    scatter.set_offsets(np.c_[data_xarray['lon'].values[time_id], data_xarray['lat'].values[time_id]])

    plt.savefig(f"./video/position_{i:04}.png", dpi=75, bbox_inches="tight")